# SHOOTING TRAINER

## Objective:
	
    Improve basketball players shooting performance, through the use of computer vision and machine learning.
    

## Background:

According to Harmon (2016) "With the rise of deep neural networks, sport prediction experts have new tools for analyzing players, match-ups, and team strategy for unique multiagent systems.(p.2). The recent advancements in computing power, image decomposition, and computer graphics have made it possiblee for players of any age to make substantial improvement in their shooting accuracy and their overall performance on the court, through the use of cutting edge technology. The technology is advancing quickly, already companies like Stats LLC. are currently utilizing high definition cameras positioned in the rafters of NBA stadiums to track every movement that occurs on the court within the game. They then, with the use of image decomposition and object tracking, analyze that data. The end result, ideally, is a predictive model that identifies the attributes of the actions on the court that lead to the highest potential for a positive outcome in any situation. One can only expect these trends to continue as the methods for collecting the data improve tremendously.

Over the last 3 years there has been a significant amount of research done regarding the use of video image tracking and professional basketball. Utilizing data gathered by Stats, researchers have been investigating everything from predicting individual’s performance, teams wins and losses, and shot results. This project, is the intial step in a larger project designed by Mark Harmon, Patrick Lucey, and Diego Klabjan in their study entitled “Predicting Shot Making in Basketball Learnt from Adversarial Multiagent Trajectories” (2017). In their research, they predict miss or made shots by looking at factors within the last 5 seconds of the play. They utilize a combination of linear regression, feed forward neural net, and convolutional neural net and measure their results using the log loss function. Their study predicted in-game made or missed shots with a 39% error.

## Technical Report: 
	
    
The process for building a machine that can improve an individual’s shooting performance is a complex issue, which deals in several different domains. The steps for that process however are the same as any other predictive model. Understand domain or domains, gather some data, preprocess data, feature extraction, fit and train a predictive model, test the results, and gather insights.Successful completion of these tasks will yield a computer machine that can predict the outcome of an individual jump shot.



In [9]:
# Read in the necessary packages for object tracking

import numpy as np
import cv2
from sys import argv
import matplotlib.pyplot as plt
%matplotlib inline

## Data:

I will be creating my own data in the form of a video of me shooting jumpshot. This gives me the advantage of completely controlling my environmnets, plus I can shoot jump shots for a couple of hours and legitimately call it work. We will be focusing on if the ball is being tracked and why. We will not be focusing on the fact that I nearly shoot an airball becuase I do not bend my knees. 

The video is in the mp4 format and entitled "tracking_video"

In [10]:
# Read in video image using opencv capture command

cap = cv2.VideoCapture('tracking_video.mp4')

# take first frame of the video
ret,frame = cap.read()

In [11]:
# setup initial location of window
r,h,c,w = 250,90,400,125  # simply hardcoded the values
track_window = (c,r,w,h)

In [12]:
# set up the ROI for tracking by adding the height and width of the object being tracked in every frame
roi = frame[r:r+h, c:c+w]

Opencv perfers the image to be in Hue, Saturation, Value coordinates as opposed to Blue, Green, Red

In [5]:
hsv_roi =  cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)  # convert image color from BGR to HSV

In [14]:
# Create an orange mask for the image, allowing the ball to be the main object in the foreground of the image.

mask = cv2.inRange(hsv_roi, np.array((9.,100.,100.)), np.array((29.,255.,255.)))

## Algorithm

Once the mask has been established and the ball has been isolated in the forground of the image, the program will process all 2700 frames of the video locating and placing the ball in the ROI in every frame. The result is the tracking of the ball as it moves through the air.

In [15]:
#plot histogram of the image based on the ditribution of the color orange and min max normalize the  
roi_hist = cv2.calcHist([hsv_roi],[0],mask,[180],[0,180])
cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

# Setup the termination criteria, either 5 iteration or move by atleast 1 pt
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 5, 1)

#check for end of video if not continue
while(1):
    ret ,frame = cap.read()

    if ret == True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)    #caluclate the relative histogram of the image object color

        # apply meanshift to get the new location of centroids in each frame
        ret, track_window = cv2.CamShift(dst, track_window, term_crit)
        
    
        
        # Draw it on image
        x,y,w,h = track_window
        img2 = cv2.rectangle(frame, (x,y), (x+w,y+h), 255,2)
        cv2.imshow('img2',img2)

        k = cv2.waitKey(60) & 0xff
        if k == 27:

            break
        else:
            cv2.imwrite(chr(k)+".jpg",img2)

    else:
        break

cv2.destroyAllWindows()
cap.release()